# Install Requirements

In [ ]:
!pip install -r requirements.txt

# Get Data

In [ ]:
# 2016
dataset_name = "ISIC_2016"
train_zip_url = "https://isic-challenge-data.s3.amazonaws.com/2016/ISBI2016_ISIC_Part1_Training_Data.zip"
test_zip_url = "https://isic-challenge-data.s3.amazonaws.com/2016/ISBI2016_ISIC_Part1_Test_Data.zip"
mask_train_zip_url = "https://isic-challenge-data.s3.amazonaws.com/2016/ISBI2016_ISIC_Part1_Training_GroundTruth.zip"
mask_test_zip_url = "https://isic-challenge-data.s3.amazonaws.com/2016/ISBI2016_ISIC_Part1_Test_GroundTruth.zip"
train_path = "data/train"
test_path = "data/test"
mask_train_path = "data/mask_train"
mask_test_path = "data/mask_test"

In [ ]:
!python data/preprocessing.py \
--dataset_name $dataset_name \
--train_zip_url $train_zip_url \
--test_zip_url $test_zip_url \
--mask_train_zip_url $mask_train_zip_url \
--mask_test_zip_url $mask_test_zip_url \
--train_path $train_path \
--test_path $test_path \
--mask_train_path $mask_train_path \
--mask_test_path $mask_test_path

# Training Models
### Parameters

In [ ]:
import shutil
import os
from utils.group_metrics import get_mean_std

In [ ]:
# basic
dataset_name = "ISIC_2016"
model_names = [
    'r2unet_cbam',
    'r2unet',
    'unet_res50',
    'unet_conv_deconv',
    'unet_attention_gate', # This model is not deterministic
    'unet_cbam',
    'unet_cbam_gate',
    'unet_pyramid_cbam_gate',
    "mcg_unet",
    'double_unet',
    ]
               
save_path = "saved_models/"
train_identifier = "usual"
multi_train = 1 # Run model for several times to estmiate std and mean metrics!
epochs = 1
batch_size = 32
img_size = (256, 256)
img_channel = 3
loss = "dice_loss"

# learning rate
lr = 1e-3
min_lr = 1e-5

# callbacks
reduce_lr_patience = 15
reduce_lr_factor = 0.1
early_stopping_p = 30

seeds = 1234
verbose = 1

# Augmentation
hair_aug_p = 0
hair_rmv_p = 0

mosaic_p = 0
cutmix_p = 0
cutmix_beta = 0

# Constant
hue_shift_limit = 1
sat_shift_limit = 0
contrast_limit = 0.1
brightness_limit = 0.1
hue_p = 0.5
contrast_p = 0.5
brightness_p = 0.5
random_rotate_p = 0.5
p_horizontal_flip = 0.5
p_vertical_flip = 0.5
p_center_crop = 0.5

metrics_names = (
    "dice_loss", 
    "dice_score",
    "focal_tversky_loss",
    "iou",
    "jaccard_loss",
    "loss",
    "val_dice_loss",
    "val_dice_score",
    "val_focal_tversky_loss",
    "val_iou",
    "val_jaccard_loss",
    "val_loss"
         )
metrics_operators = (min, max, min, max, min, min, min, max, min, max, min, min)

In [ ]:
def save_print_mean_std(save_path, dataset_name, model, train_identifier, multi_train, metrics_names, metrics_operators, model_path):
    try:
        csv_addresses = [
 os.path.join(save_path, dataset_name, model, f"{train_identifier}_{n}", "exp_1" , "csv_logger_train.csv") for n in range(multi_train)   
]
        metrics = get_mean_std(csv_addresses, 
                       arguments=metrics_names,
                       operators=metrics_operators,
                       save_path=os.path.join(save_path, dataset_name, model, f"{train_identifier}_mean_std_metrics.csv")
                      )
        for name, metric in metrics.items():
            print(f"[INFO] Model: {model} == > {name}: {metric}")
    except Exception as e:
        print(f"[ERROR] {e} raised for {model_path}")

In [ ]:
# Ensure reproducibility
# !PYTHONHASHSEED=0
# !TF_DETERMINISTIC_OPS=0
# !TF_CUDNN_DETERMINISTIC=0

In [ ]:
import shutil
import os
from utils.group_metrics import get_mean_std

for model in model_names:
    model_path = os.path.join(save_path, dataset_name, model)
    for n in range(multi_train):
        seed = seeds + n
        saved_path_name = f"{train_identifier}_{n}"
        iter_model = os.path.join(model_path, saved_path_name)
        print(f"[INFO] Removing old model checkpoints: {iter_model}!!")
        shutil.rmtree(iter_model) if os.path.isdir(iter_model) else print(f'[INFO] {iter_model} does not exist!')
        !python train.py \
        --model $model --dataset_name $dataset_name \
        --save_path $save_path \
        --save_path_name $saved_path_name \
        --epochs $epochs \
        --lr $lr \
        --min_lr $min_lr \
        --reduce_lr_patience $reduce_lr_patience \
        --reduce_lr_factor $reduce_lr_factor \
        --early_stopping_p $early_stopping_p \
        --hair_aug_p $hair_aug_p \
        --hair_rmv_p $hair_rmv_p \
        --random_rotate_p $random_rotate_p \
        --p_horizontal_flip $p_horizontal_flip \
        --p_vertical_flip $p_vertical_flip \
        --p_center_crop $p_center_crop \
        --mosaic_p $mosaic_p \
        --cutmix_p $cutmix_p \
        --cutmix_beta $cutmix_beta \
        --seed $seed
    
        print(f"\n-----------------------------train {n} is done! for model: {model} ----------------------------------\n")
        print(f"\n-----------------------------train {n} is done! for model: {model} ----------------------------------\n")
        print(f"\n-----------------------------train {n} is done! for model: {model} ----------------------------------\n")
    save_print_mean_std(save_path, dataset_name, model, train_identifier, multi_train, metrics_names, metrics_operators, model_path)

## With cutmix and Mosaic

In [ ]:
train_identifier = "usual_cutmix_mosaic"

# Augmentation
hair_aug_p = 0
hair_rmv_p = 0

mosaic_p = 0.5
cutmix_p = 0.5
cutmix_beta = 1


In [ ]:
for model in model_names:
    model_path = os.path.join(save_path, dataset_name, model)
    for n in range(multi_train):
        seed = seeds + n
        saved_path_name = f"{train_identifier}_{n}"
        iter_model = os.path.join(model_path, saved_path_name)
        print(f"[INFO] Removing old model checkpoints: {iter_model}!!")
        shutil.rmtree(iter_model) if os.path.isdir(iter_model) else print(f'[INFO] {iter_model} does not exist!')
        !python train.py \
        --model $model --dataset_name $dataset_name \
        --save_path $save_path \
        --save_path_name $saved_path_name \
        --epochs $epochs \
        --lr $lr \
        --min_lr $min_lr \
        --reduce_lr_patience $reduce_lr_patience \
        --reduce_lr_factor $reduce_lr_factor \
        --early_stopping_p $early_stopping_p \
        --hair_aug_p $hair_aug_p \
        --hair_rmv_p $hair_rmv_p \
        --random_rotate_p $random_rotate_p \
        --p_horizontal_flip $p_horizontal_flip \
        --p_vertical_flip $p_vertical_flip \
        --p_center_crop $p_center_crop \
        --mosaic_p $mosaic_p \
        --cutmix_p $cutmix_p \
        --cutmix_beta $cutmix_beta \
        --seed $seed
    
        print(f"\n-----------------------------train {n} is done! for model: {model} ----------------------------------\n")
        print(f"\n-----------------------------train {n} is done! for model: {model} ----------------------------------\n")
        print(f"\n-----------------------------train {n} is done! for model: {model} ----------------------------------\n")
    save_print_mean_std(save_path, dataset_name, model, train_identifier, multi_train, metrics_names, metrics_operators, model_path)

# With Cutmix & Mosaic & hair aug

In [ ]:
train_identifier = "usual_cutmix_mosaic_hair_aug"

# Augmentation
hair_aug_p = 0.5
hair_rmv_p = 0

mosaic_p = 0.5
cutmix_p = 0.5
cutmix_beta = 1

In [ ]:
for model in model_names:
    model_path = os.path.join(save_path, dataset_name, model)
    for n in range(multi_train):
        seed = seeds + n
        saved_path_name = f"{train_identifier}_{n}"
        iter_model = os.path.join(model_path, saved_path_name)
        print(f"[INFO] Removing old model checkpoints: {iter_model}!!")
        shutil.rmtree(iter_model) if os.path.isdir(iter_model) else print(f'[INFO] {iter_model} does not exist!')
        !python train.py \
        --model $model --dataset_name $dataset_name \
        --save_path $save_path \
        --save_path_name $saved_path_name \
        --epochs $epochs \
        --lr $lr \
        --min_lr $min_lr \
        --reduce_lr_patience $reduce_lr_patience \
        --reduce_lr_factor $reduce_lr_factor \
        --early_stopping_p $early_stopping_p \
        --hair_aug_p $hair_aug_p \
        --hair_rmv_p $hair_rmv_p \
        --random_rotate_p $random_rotate_p \
        --p_horizontal_flip $p_horizontal_flip \
        --p_vertical_flip $p_vertical_flip \
        --p_center_crop $p_center_crop \
        --mosaic_p $mosaic_p \
        --cutmix_p $cutmix_p \
        --cutmix_beta $cutmix_beta \
        --seed $seed
    
        print(f"\n-----------------------------train {n} is done! for model: {model} ----------------------------------\n")
        print(f"\n-----------------------------train {n} is done! for model: {model} ----------------------------------\n")
        print(f"\n-----------------------------train {n} is done! for model: {model} ----------------------------------\n")
    save_print_mean_std(save_path, dataset_name, model, train_identifier, multi_train, metrics_names, metrics_operators, model_path)

# With Cutmix & Mosaic & hair rmv

In [ ]:
train_identifier = "usual_cutmix_mosaic_hair_rmv"

# Augmentation
hair_aug_p = 0
hair_rmv_p = 0.5

mosaic_p = 0.5
cutmix_p = 0.5
cutmix_beta = 1

In [ ]:
for model in model_names:
    model_path = os.path.join(save_path, dataset_name, model)
    for n in range(multi_train):
        seed = seeds + n
        saved_path_name = f"{train_identifier}_{n}"
        iter_model = os.path.join(model_path, saved_path_name)
        print(f"[INFO] Removing old model checkpoints: {iter_model}!!")
        shutil.rmtree(iter_model) if os.path.isdir(iter_model) else print(f'[INFO] {iter_model} does not exist!')
        !python train.py \
        --model $model --dataset_name $dataset_name \
        --save_path $save_path \
        --save_path_name $saved_path_name \
        --epochs $epochs \
        --lr $lr \
        --min_lr $min_lr \
        --reduce_lr_patience $reduce_lr_patience \
        --reduce_lr_factor $reduce_lr_factor \
        --early_stopping_p $early_stopping_p \
        --hair_aug_p $hair_aug_p \
        --hair_rmv_p $hair_rmv_p \
        --random_rotate_p $random_rotate_p \
        --p_horizontal_flip $p_horizontal_flip \
        --p_vertical_flip $p_vertical_flip \
        --p_center_crop $p_center_crop \
        --mosaic_p $mosaic_p \
        --cutmix_p $cutmix_p \
        --cutmix_beta $cutmix_beta \
        --seed $seed
    
        print(f"\n-----------------------------train {n} is done! for model: {model} ----------------------------------\n")
        print(f"\n-----------------------------train {n} is done! for model: {model} ----------------------------------\n")
        print(f"\n-----------------------------train {n} is done! for model: {model} ----------------------------------\n")
    save_print_mean_std(save_path, dataset_name, model, train_identifier, multi_train, metrics_names, metrics_operators, model_path)

_:)_